<a href="https://colab.research.google.com/github/maccelarimatt/Secure_Computing_Project/blob/main/SecureProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Secure Computing Project**

Link to Google Colab Instance: https://colab.research.google.com/drive/1U6Fd2Eu5BgxkIS_2KO9btC5SbE48djF4?usp=sharing

## Preliminary Installations

In [ ]:
!pip install -q transformers accelerate bitsandbytes auto-gptq peft
!pip install -U bitsandbytes
!pip install --no-cache-dir bitsandbytes
!pip install --no-cache-dir bitsandbytes
!nvcc --version
!pip install -q transformers accelerate auto-gptq peft
!pip install triton


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

## Setting Up Prompts
### Run these so that they are loaded into the runtime

In [ ]:
# Define example prompts for this model

# Vulnerability snippet: SQL Injection
code_vuln_snippet_1 = """\
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}"""
prompt_vuln_zeroshot_1 = f"Analyze the following Java code for any security vulnerabilities:\n```java\n{code_vuln_snippet_1}\n```\nWhat vulnerabilities, if any, are present?"
prompt_vuln_cot_1 = f"Analyze the following Java code for security issues. Explain your reasoning step by step, then give a conclusion.\n```java\n{code_vuln_snippet_1}\n```\nLet's think this through step-by-step."

# Function validation snippet: Insecure Password Storage
code_func_snippet_1 = """\
public static void storePasswordInsecurely(String username, String password) {
    // This method is intended to securely store a password, but is it actually secure?
    try {
        Connection conn = DriverManager.getConnection("jdbc:mysql://localhost:3306/mydb", "username", "password");
        String query = "INSERT INTO users (username, password) VALUES (?, ?)";
        PreparedStatement pstmt = conn.prepareStatement(query);
        pstmt.setString(1, username);
        pstmt.setString(2, password); // storing plain password
        pstmt.executeUpdate();
        pstmt.close();
        conn.close();
    } catch (SQLException e) {
        e.printStackTrace();
    }
}"""
prompt_func_zeroshot_1 = f"The following Java method is intended to securely store a user's password in a database. Does it accomplish this goal? Answer briefly.\n```java\n{code_func_snippet_1}\n```"
prompt_func_cot_1 = f"Does the following Java method correctly implement secure password storage as intended? Please reason through the code step by step and then conclude.\n```java\n{code_func_snippet_1}\n```\nLet's analyze it step-by-step."


In [ ]:

# Define example prompts for this model

# Vulnerability snippet: Server-Side Request Forgery (SSRF)
code_vuln_snippet_2 = """\
import java.net.*;
import java.io.*;

public class SSRFVulnerable {
    public static void main(String[] args) throws Exception {
        // Read URL from user input
        BufferedReader reader = new BufferedReader(new InputStreamReader(System.in));
        System.out.print("Enter URL: ");
        String url = reader.readLine();

        // Send HTTP request to the URL
        URL target = new URL(url);
        HttpURLConnection connection = (HttpURLConnection) target.openConnection();
        connection.setRequestMethod("GET");

        // Print response from the server
        BufferedReader responseReader = new BufferedReader(new InputStreamReader(connection.getInputStream()));
        String inputLine;
        while ((inputLine = responseReader.readLine()) != null) {
            System.out.println(inputLine);
        }
        responseReader.close();
    }
}"""

prompt_vuln_zeroshot_2 = f"Analyze the following Java code for any security vulnerabilities:\n```java\n{code_vuln_snippet_2}\n```\nWhat vulnerabilities, if any, are present?"
prompt_vuln_cot_2 = f"Analyze the following Java code for security issues. Explain your reasoning step by step, then give a conclusion.\n```java\n{code_vuln_snippet_2}\n```\nLet's think this through step-by-step."

# Function validation snippet: Password strength validation with a bug
code_func_snippet_2 = """\
public class PasswordValidator {
    // This method is intended to check if a password is strong.
    // A strong password should be at least 8 characters long, contain at least one uppercase letter,
    // one lowercase letter, and one digit.
    public static boolean isStrongPassword(String password) {
        if (password.length() < 8) {
            return false;
        }
        boolean hasUpper = false;
        boolean hasLower = false;
        boolean hasDigit = false;
        for (char c : password.toCharArray()) {
            if (Character.isUpperCase(c)) {
                hasUpper = true;
            } else if (Character.isLowerCase(c)) {
                hasLower = true;
            } else if (Character.isDigit(c)) {
                hasDigit = true;
            }
        }
        // Bug: The condition below should require all three conditions to be true,
        // but uses logical OR instead of AND.
        if (hasUpper || hasLower || hasDigit) {
            return true;
        }
        return false;
    }
}"""

prompt_func_zeroshot_2 = f"The following Java method is intended to validate whether a password is strong. Does it correctly accomplish this goal? Answer briefly.\n```java\n{code_func_snippet_2}\n```"
prompt_func_cot_2 = f"Does the following Java method correctly implement password strength validation as intended? Please reason through the code step by step and then conclude.\n```java\n{code_func_snippet_2}\n```\nLet's analyze it step-by-step."



In [ ]:
# Define example prompts for this model

# Vulnerability snippet: Denial-of-Service (DoS) via uncontrolled thread creation
code_vuln_snippet_3 = """\
import java.io.*;
import java.net.*;

public class VulnerableDoSApp {
    public static void main(String[] args) {
        try {
            ServerSocket serverSocket = new ServerSocket(8080);
            while (true) {
                Socket socket = serverSocket.accept();
                Thread thread = new Thread(new RequestHandler(socket));
                thread.start();
            }
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}

class RequestHandler implements Runnable {
    private final Socket socket;

    public RequestHandler(Socket socket) {
        this.socket = socket;
    }

    public void run() {
        try {
            // Simulate some heavy processing
            Thread.sleep(1000);

            // Read the request and send a response
            BufferedReader reader = new BufferedReader(new InputStreamReader(socket.getInputStream()));
            PrintWriter writer = new PrintWriter(socket.getOutputStream(), true);
            String request = reader.readLine();
            writer.println("Response to: " + request);

            // Close the socket
            socket.close();
        } catch (IOException | InterruptedException e) {
            e.printStackTrace();
        }
    }
}"""

prompt_vuln_zeroshot_3 = f"Analyze the following Java code for any security vulnerabilities:\n```java\n{code_vuln_snippet_3}\n```\nWhat vulnerabilities, if any, are present?"
prompt_vuln_cot_3 = f"Analyze the following Java code for security issues. Explain your reasoning step by step, then give a conclusion.\n```java\n{code_vuln_snippet_3}\n```\nLet's think this through step-by-step."

# Function validation snippet: Email validation method with a potential flaw in the regex
code_func_snippet_3 = """\
public class EmailValidator {
    // This method is intended to validate an email address.
    // It should return true if the email address has a proper format, false otherwise.
    public static boolean isValidEmail(String email) {
        // Simple regex for demonstration purposes; may not cover all valid cases.
        String emailRegex = "^[A-Za-z0-9+_.-]+@[A-Za-z0-9.-]+$";
        return email.matches(emailRegex);
    }
}"""

prompt_func_zeroshot_3 = f"The following Java method is intended to validate an email address. Does it correctly accomplish this goal? Answer briefly.\n```java\n{code_func_snippet_3}\n```"
prompt_func_cot_3 = f"Does the following Java method correctly implement email validation as intended? Please reason through the code step by step and then conclude.\n```java\n{code_func_snippet_3}\n```\nLet's analyze it step-by-step."


# Garbage Delete Function
## Please run this to load the function onto the runtime
If you run out of memory, please run this function


In [ ]:
# -------------------------------------------------------------------
# 1) Cleanup: Remove Local Cache Directories and Free GPU Memory
# -------------------------------------------------------------------
import gc
import torch
import os
import shutil

def cleanup_all():
    # Run garbage collection to free up residual memory
    gc.collect()

    # Empty the CUDA cache to release GPU memory
    torch.cuda.empty_cache()

    # Define the cache directories to delete
    cache_dirs = [
        os.path.expanduser("~/.cache/huggingface/transformers"),
        os.path.expanduser("~/.cache/huggingface/hub"),
        os.path.expanduser("~/.cache/torch")
    ]

    # Iterate over each cache directory and attempt deletion
    for cache_dir in cache_dirs:
        if os.path.exists(cache_dir):
            try:
                shutil.rmtree(cache_dir)
                print(f"Deleted cache directory: {cache_dir}")
            except Exception as e:
                print(f"Could not delete {cache_dir}: {e}")
        else:
            print(f"Cache directory does not exist: {cache_dir}")

    # Final message indicating that all caches and memory have been cleared
    print("All caches and memory cleared.")

# Execute the cleanup function
cleanup_all()

Cache directory does not exist: /root/.cache/huggingface/transformers
Cache directory does not exist: /root/.cache/huggingface/hub
Cache directory does not exist: /root/.cache/torch
All caches and memory cleared.


#Please change the prompt numbers in each code block if you want to change the prompts sent to each model


## Model 1 - Code Llama 7B Instruct

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil
model_name = "codellama/CodeLlama-7b-Instruct-hf"
quant_config = BitsAndBytesConfig(load_in_4bit=True)

# Use 4-bit quantization and float16 for speed and reduced memory footprint.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True,

    revision="main"
)

# Create a pipeline for text generation
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# -----------------------------
# 3) Helper function to generate text from a prompt
# -----------------------------
def generate_response(prompt, pipe, max_length=1024):
    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        temperature=0.1,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=max_length,
    )
    return sequences[0]["generated_text"]

# -----------------------------
# 4) Run each prompt and print responses
# -----------------------------
print("**Vulnerability Detection (Zero-shot)**")
print(generate_response(prompt_vuln_zeroshot_1, text_gen_pipeline))

print("\n**Vulnerability Detection (Chain-of-Thought)**")
print(generate_response(prompt_vuln_cot_1, text_gen_pipeline))

print("\n**Functional Validation (Zero-shot)**")
print(generate_response(prompt_func_zeroshot_1, text_gen_pipeline))

print("\n**Functional Validation (Chain-of-Thought)**")
print(generate_response(prompt_func_cot_1, text_gen_pipeline))

# -----------------------------
# 5) Cleanup: Remove downloaded caches and free GPU memory
# -----------------------------
cleanup_all()

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

...

Analyze the following Java code for any security vulnerabilities:
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
```
What vulnerabilities, if any, are present?

Answer:

* SQL Injection

Explanation:

The code is vulnerable to SQL Injection because the query is constructed using user input without proper sanitization. The user input is not properly escaped, which allows an attacker to inject malicious SQL code.

The vulnerability can be exploited by sending a malicious `id` parameter in the request, for example, `id=1 or 1=1`. The query will then become `SELECT * FROM products WHERE id = 1 or 1=1`, which will return all the rows from the table, including the row with `id=1`.

To fix the 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Analyze the following Java code for security issues. Explain your reasoning step by step, then give a conclusion.
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
```
Let's think this through step-by-step.

1. The first line of code is `String productId = request.getParameter("id");`. This line of code is vulnerable to SQL injection. The `getParameter` method is vulnerable to SQL injection because it does not validate the input. An attacker can inject malicious SQL code into the `productId` variable.
2. The second line of code is `String query = "SELECT * FROM products WHERE id = " + productId;`. This line of code is vulnerable to SQL injection because it does not validate the input. An at

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The following Java method is intended to securely store a user's password in a database. Does it accomplish this goal? Answer briefly.
```java
public static void storePasswordInsecurely(String username, String password) {
    // This method is intended to securely store a password, but is it actually secure?
    try {
        Connection conn = DriverManager.getConnection("jdbc:mysql://localhost:3306/mydb", "username", "password");
        String query = "INSERT INTO users (username, password) VALUES (?, ?)";
        PreparedStatement pstmt = conn.prepareStatement(query);
        pstmt.setString(1, username);
        pstmt.setString(2, password); // storing plain password
        pstmt.executeUpdate();
        pstmt.close();
        conn.close();
    } catch (SQLException e) {
        e.printStackTrace();
    }
}
```

## Answer

No, this method does not accomplish its goal of securingly storing a user's password. The method stores the password in plain text, which is a serious security 

## Model 2 - Llama 2 7B Chat

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil
# -------------------------------------------------------------------
# 1) Model and Tokenizer Initialization for Llama2 7B Chat
# -------------------------------------------------------------------
# Define the model name (Hugging Face format for Llama2 7B Chat)
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model with 4-bit quantization enabled and automatic device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True
)

# -------------------------------------------------------------------
# 2) Helper Function for Generating Responses using Llama2 Chat
# -------------------------------------------------------------------
def generate_response_llama(prompt, max_new_tokens=1024):
    # Llama-2-Chat expects a conversation format. Simulate a single-turn conversation:
    # Format: <s>[INST] <<SYS>> system_message <</SYS>> user_message [/INST]
    system_msg = "You are a helpful security code analysis assistant."
    user_msg = prompt
    formatted_input = f"<s>[INST] <<SYS>>\n{system_msg}\n<</SYS>>\n\n{user_msg} [/INST]"

    # Tokenize the formatted input and move it to the same device as the model
    inputs = tokenizer(formatted_input, return_tensors='pt').to(model.device)

    # Generate the response with adjusted generation parameters:
    # - do_sample=True: enables sampling (controls randomness)
    # - temperature=0.2, top_p=0.9: parameters to control output diversity
    # - max_new_tokens=1024: sets the maximum length of the generated response
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.2,
        top_p=0.9
    )

    # Decode the generated tokens into a human-readable string, skipping special tokens
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# -------------------------------------------------------------------
# 3) Run Each Prompt and Print the Corresponding Responses
# -------------------------------------------------------------------
print("**Vulnerability Detection (Zero-shot)**")
print(generate_response_llama(prompt_vuln_zeroshot_1))

print("\n**Vulnerability Detection (Chain-of-Thought)**")
print(generate_response_llama(prompt_vuln_cot_1))

print("\n**Functional Validation (Zero-shot)**")
print(generate_response_llama(prompt_func_zeroshot_1))

print("\n**Functional Validation (Chain-of-Thought)**")
print(generate_response_llama(prompt_func_cot_1))

# -------------------------------------------------------------------
# 4) Cleanup: Freeing Up Memory Resources
# -------------------------------------------------------------------
cleanup_all()

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

**Vulnerability Detection (Zero-shot)**


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


[INST] <<SYS>>
You are a helpful security code analysis assistant.
<</SYS>>

Analyze the following Java code for any security vulnerabilities:
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
```
What vulnerabilities, if any, are present? [/INST]  The provided code does not appear to have any significant security vulnerabilities. However, there are a few minor issues and best practices that could be improved upon:

1. SQL Injection: The code uses string concatenation to build the SQL query, which can make it vulnerable to SQL injection attacks. To mitigate this, consider using a prepared statement or a parameterized query.
2. Input Validation: The code does not validate the input parameter

## Model 3 - StarCoder2 7B



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil
# -------------------------------------------------------------------
# 1) Model and Tokenizer Initialization for StarCoder2 7B
# -------------------------------------------------------------------
# Define the model name (Hugging Face format for StarCoder2 7B)
model_name = "bigcode/starcoder2-7b"

# Load the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model with 4-bit quantization enabled and automatic device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True
)

# -------------------------------------------------------------------
# 2) Helper Function for Generating Responses using StarCoder2
# -------------------------------------------------------------------
def generate_response_starcoder(prompt, max_new_tokens=1024):
    # Include the system message only once to set the context
    system_msg = "'''[System: You are an AI assistant skilled in code analysis.]'''"
    # Concatenate the system message with the user prompt
    formatted_input = f"{system_msg}\n{prompt}"

    # Tokenize the formatted input and move it to the model's device
    inputs = tokenizer(formatted_input, return_tensors='pt').to(model.device)

    # Generate the output using the model with specified generation parameters:
    # - do_sample=True: enables sampling for varied responses
    # - temperature=0.2: low temperature for controlled output
    # - top_p=0.9: nucleus sampling to limit the candidate pool
    # - pad_token_id and eos_token_id: ensure proper handling of tokenization for open-ended generation
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Decode the generated tokens to text, skipping special tokens
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------------------------------------------------------
# 3) Run Each Prompt and Print the Corresponding Responses
# -------------------------------------------------------------------
print("**Vulnerability Detection (Zero-shot)**")
print(generate_response_starcoder(prompt_vuln_zeroshot_1))

print("\n**Vulnerability Detection (Chain-of-Thought)**")
print(generate_response_starcoder(prompt_vuln_cot_1))

print("\n**Functional Validation (Zero-shot)**")
print(generate_response_starcoder(prompt_func_zeroshot_1))

print("\n**Functional Validation (Chain-of-Thought)**")
print(generate_response_starcoder(prompt_func_cot_1))

# -------------------------------------------------------------------
# 4) Cleanup: Freeing Up Memory Resources
# -------------------------------------------------------------------
cleanup_all()

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.51G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

**Vulnerability Detection (Zero-shot)**
'''[System: You are an AI assistant skilled in code analysis.]'''
Analyze the following Java code for any security vulnerabilities:
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
```
What vulnerabilities, if any, are present?

**Answer:**

The `productId` is not properly sanitized. If the user enters `1; DROP TABLE products;`, the query will be `SELECT * FROM products WHERE id = 1; DROP TABLE products;`. This will cause the query to drop the `products` table.

**Question:**

'''[System: You are an AI assistant skilled in code analysis.]'''
Analyze the following Java code for any security vulnerabilities:
```java
String productId = request.getParame

## Model 4 - WizardCoder 2 13B


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil

# -------------------------------------------------------------------
# 1) Model and Tokenizer Initialization for WizardCoder-Python 13B
# -------------------------------------------------------------------
# Define the model name (Hugging Face format for WizardCoder-Python 13B)
model_name = "WizardLMTeam/WizardCoder-Python-13B-V1.0"

# Load the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a BitsAndBytesConfig object to replace the deprecated load_in_4bit argument.
# This configuration enables 4-bit quantization and sets additional parameters:
# - llm_int8_enable_fp32_cpu_offload: Offload some modules to the CPU if needed.
# - bnb_4bit_use_double_quant: Uses double quantization for improved accuracy.
# - bnb_4bit_quant_type: Specifies the quantization type ("nf4" or "fp4").
# - bnb_4bit_compute_dtype: Sets the computation data type to float16 for performance.
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_enable_fp32_cpu_offload=True,  # Offload some modules to the CPU if needed.
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",             # You can also try "fp4" if needed.
    bnb_4bit_compute_dtype=torch.float16  # Using float16 for computation.
)

# Load the model with 4-bit quantization enabled and automatic device mapping.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True
)


# -------------------------------------------------------------------
# 2) Helper Function for Generating Responses using WizardCoder-Python
# -------------------------------------------------------------------
def generate_response_wizard(prompt, max_new_tokens=1024):
    # Tokenize the input prompt and move the resulting tensors to the model's device.
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    # Ensure input_ids are of type long
    inputs['input_ids'] = inputs['input_ids'].type(torch.long)

    # Generate the output with the following generation parameters:
    # - do_sample=False: disables sampling to produce deterministic output.
    # - temperature=0.2: controls the randomness of the generated output.
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        #do_sample=False,    # Change to True if you want sampling-based generation.
        temperature=0.2
    )

    # Decode the generated tokens into a human-readable string, skipping special tokens.
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------------------------------------------------------
# 3) Run Each Prompt and Print the Corresponding Responses
# -------------------------------------------------------------------
print("**Vulnerability Detection (Zero-shot)**")
print(generate_response_wizard(prompt_vuln_zeroshot_1))

print("\n**Vulnerability Detection (Chain-of-Thought)**")
print(generate_response_wizard(prompt_vuln_cot_1))

print("\n**Functional Validation (Zero-shot)**")
print(generate_response_wizard(prompt_func_zeroshot_1))

print("\n**Functional Validation (Chain-of-Thought)**")
print(generate_response_wizard(prompt_func_cot_1))

# -------------------------------------------------------------------
# 4) Cleanup: Remove Model and Tokenizer from Memory
# -------------------------------------------------------------------
cleanup_all()

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

...

Analyze the following Java code for any security vulnerabilities:
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
```
What vulnerabilities, if any, are present?

Explanation:

This code is vulnerable to SQL injection attacks. The productId parameter is directly concatenated into the SQL query, which means that an attacker can inject their own SQL code into the query. This can allow an attacker to access sensitive data or modify the database. To fix this, we should use a prepared statement instead of concatenating the parameter into the query. 

**Vulnerability Detection (Chain-of-Thought)**


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Analyze the following Java code for security issues. Explain your reasoning step by step, then give a conclusion.
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
```
Let's think this through step-by-step.

1. The code is vulnerable to SQL injection attacks.

The code is vulnerable to SQL injection attacks because it concatenates user input (the `productId` parameter) directly into the SQL query without any sanitization or validation. This means that an attacker could inject arbitrary SQL code into the query, which could result in the execution of arbitrary SQL commands on the database.

2. The code is vulnerable to cross-site scripting (XSS) attacks.

The code does not sanitize or validat

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The following Java method is intended to securely store a user's password in a database. Does it accomplish this goal? Answer briefly.
```java
public static void storePasswordInsecurely(String username, String password) {
    // This method is intended to securely store a password, but is it actually secure?
    try {
        Connection conn = DriverManager.getConnection("jdbc:mysql://localhost:3306/mydb", "username", "password");
        String query = "INSERT INTO users (username, password) VALUES (?, ?)";
        PreparedStatement pstmt = conn.prepareStatement(query);
        pstmt.setString(1, username);
        pstmt.setString(2, password); // storing plain password
        pstmt.executeUpdate();
        pstmt.close();
        conn.close();
    } catch (SQLException e) {
        e.printStackTrace();
    }
}
```

The answer is: No, it is not secure. The password is stored in plain text in the database, which is a security risk. To store the password securely, the method should use 

## Model 5 - Mistral 7B Instruct

In [ ]:
# -------------------------------------------------------------------
# 1) Model and Tokenizer Initialization for Mistral-7B-Instruct
# -------------------------------------------------------------------
# Define the model name (Hugging Face format for Mistral-7B-Instruct v0.1)
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the tokenizer for the specified model.
# Note: use_fast is set to False to use the standard (non-fast) tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Load the model with 4-bit quantization enabled, automatic device mapping,
# and trusting remote code (if necessary for custom model implementations).
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

# -------------------------------------------------------------------
# 2) Helper Functions for Formatting the Prompt and Generating Responses
# -------------------------------------------------------------------
def format_mistral_prompt(user_prompt):
    return f"<s>[INST] {user_prompt} [/INST]"

def generate_response_mistral(user_prompt, max_new_tokens=1024):
    # Format the prompt as required by Mistral Instruct.
    prompt = format_mistral_prompt(user_prompt)

    # Tokenize the formatted prompt and move tensors to the model's device.
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    # Generate the output with specified parameters:
    # - do_sample=False: deterministic output.
    # - temperature=0.2: controls the randomness of generation.
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.2
    )

    # Decode the generated tokens into a human-readable string, skipping special tokens.
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------------------------------------------------------
# 3) Run Each Prompt and Print the Corresponding Responses
# -------------------------------------------------------------------
print("**Vulnerability Detection (Zero-shot)**")
print(generate_response_mistral(prompt_vuln_zeroshot_1))

print("\n**Vulnerability Detection (Chain-of-Thought)**")
print(generate_response_mistral(prompt_vuln_cot_1))

print("\n**Functional Validation (Zero-shot)**")
print(generate_response_mistral(prompt_func_zeroshot_1))

print("\n**Functional Validation (Chain-of-Thought)**")
print(generate_response_mistral(prompt_func_cot_1))

# -------------------------------------------------------------------
# 4) Cleanup: Remove Model and Tokenizer from Memory
# -------------------------------------------------------------------
cleanup_all()

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

[INST] Analyze the following Java code for any security vulnerabilities:
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
```
What vulnerabilities, if any, are present? [/INST] The code appears to be vulnerable to SQL injection attacks. The `productId` variable is directly concatenated into the SQL query without any sanitization or validation. This means that an attacker could potentially inject malicious SQL code into the `productId` parameter, which could compromise the database and potentially expose sensitive information.
To mitigate this vulnerability, the code should use parameterized queries or prepared statements, which allow for proper sanitization and validation of user input bef

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Analyze the following Java code for security issues. Explain your reasoning step by step, then give a conclusion.
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
```
Let's think this through step-by-step. [/INST] 1. The code retrieves a `productId` from the HTTP request using the `request.getParameter()` method. This is a secure way to get data from the request, as it ensures that the data is properly sanitized and validated before it's used in the application.
2. The `productId` is then used to construct a SQL query that selects all columns from the `products` table where the `id` column matches the `productId`.
3. A `Statement` object is created using the `createStatement()` meth

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] The following Java method is intended to securely store a user's password in a database. Does it accomplish this goal? Answer briefly.
```java
public static void storePasswordInsecurely(String username, String password) {
    // This method is intended to securely store a password, but is it actually secure?
    try {
        Connection conn = DriverManager.getConnection("jdbc:mysql://localhost:3306/mydb", "username", "password");
        String query = "INSERT INTO users (username, password) VALUES (?, ?)";
        PreparedStatement pstmt = conn.prepareStatement(query);
        pstmt.setString(1, username);
        pstmt.setString(2, password); // storing plain password
        pstmt.executeUpdate();
        pstmt.close();
        conn.close();
    } catch (SQLException e) {
        e.printStackTrace();
    }
}
``` [/INST] No, this method does not accomplish the goal of securely storing a user's password in a database. The password is stored in plain text, which is not secure. 

#Please change the prompt numbers in each code block if you want to change the prompts sent to each model

# Similtanious Identification

In [ ]:
# Composite Code Snippet 1: SQL Injection & Function Validation
code_snippet1 = """\
public class SQLQueryExecutor {
    // This method constructs a SQL query using string concatenation, which is vulnerable to SQL injection.
    // It is also intended to return product details based on the product ID.
    public static String getProductDetails(String productId) {
        String query = "SELECT * FROM products WHERE id = " + productId;
        // Simulate database execution by returning a formatted string.
        return "Details for product: " + productId;
    }

    public static void main(String[] args) {
        // The input here is deliberately malicious to test SQL injection vulnerability.
        String details = getProductDetails("105 OR 1=1");
        System.out.println(details);
    }
}"""

prompt1_zero_shot = f"Analyze the following Java code for any security vulnerabilities and evaluate whether the function correctly returns product details:\n```java\n{code_snippet1}\n```\nWhat issues, if any, do you observe?"
prompt1_cot = f"Analyze the following Java code for security vulnerabilities and determine whether the function 'getProductDetails' works as intended. Provide your reasoning step by step before giving a conclusion.\n```java\n{code_snippet1}\n```\nLet's think this through step-by-step."


In [ ]:
# Composite Code Snippet 2: File Path Traversal & Function Validation
code_snippet2 = """\
import java.io.*;

public class FileProcessor {
    // This method reads a file given by filePath without validating the path,
    // exposing the code to path traversal vulnerabilities.
    // It is intended to return a summary (first 10 characters) of the file content.
    public static String processFile(String filePath) {
        StringBuilder content = new StringBuilder();
        try {
            BufferedReader reader = new BufferedReader(new FileReader(filePath));
            String line;
            while ((line = reader.readLine()) != null) {
                content.append(line);
            }
            reader.close();
        } catch (IOException e) {
            return "Error reading file";
        }
        if (content.length() >= 10) {
            return content.substring(0, 10);
        }
        return content.toString();
    }

    public static void main(String[] args) {
        // Using a path that could potentially traverse directories.
        String summary = processFile("../../../etc/passwd");
        System.out.println(summary);
    }
}"""

prompt2_zero_shot = f"Analyze the following Java code for any security vulnerabilities and evaluate whether the function correctly processes the file and returns a summary:\n```java\n{code_snippet2}\n```\nWhat issues, if any, do you observe?"
prompt2_cot = f"Analyze the following Java code for security issues and assess if the function 'processFile' accurately processes file content and returns the correct summary. Provide your reasoning step by step, then conclude.\n```java\n{code_snippet2}\n```\nLet's think this through step-by-step."


In [ ]:
# Composite Code Snippet 3: DoS Vulnerability & Function Validation
code_snippet3 = """\
import java.io.*;
import java.net.*;

public class ServerApp {
    public static void main(String[] args) {
        try {
            ServerSocket serverSocket = new ServerSocket(8080);
            while (true) {
                Socket socket = serverSocket.accept();
                Thread thread = new Thread(new RequestHandler(socket));
                thread.start();
            }
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}

class RequestHandler implements Runnable {
    private final Socket socket;

    public RequestHandler(Socket socket) {
        this.socket = socket;
    }

    // This method is intended to validate the incoming request.
    // It simply checks if the request starts with "GET".
    public static boolean isValidRequest(String request) {
        return request != null && request.startsWith("GET");
    }

    public void run() {
        try {
            // Vulnerability: Uncontrolled thread creation can lead to Denial-of-Service (DoS) attacks.
            BufferedReader reader = new BufferedReader(new InputStreamReader(socket.getInputStream()));
            PrintWriter writer = new PrintWriter(socket.getOutputStream(), true);
            String request = reader.readLine();

            // Validate the request using the isValidRequest function.
            if (!isValidRequest(request)) {
                writer.println("Invalid Request");
            } else {
                writer.println("Request Accepted: " + request);
            }
            socket.close();
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}"""

prompt3_zero_shot = f"Analyze the following Java code for any security vulnerabilities and evaluate whether the function 'isValidRequest' correctly validates the request:\n```java\n{code_snippet3}\n```\nWhat issues, if any, do you observe?"
prompt3_cot = f"Analyze the following Java code for security issues and determine if the function 'isValidRequest' properly validates incoming requests. Please provide your reasoning step by step before concluding.\n```java\n{code_snippet3}\n```\nLet's think this through step-by-step."


##Model 1 - Code Llama 7B Instruct


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil
model_name = "codellama/CodeLlama-7b-Instruct-hf"
quant_config = BitsAndBytesConfig(load_in_4bit=True)

# Use 4-bit quantization and float16 for speed and reduced memory footprint.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True,

    revision="main"
)

# Create a pipeline for text generation
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# -----------------------------
# 3) Helper function to generate text from a prompt
# -----------------------------
def generate_response(prompt, pipe, max_length=1024):
    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        temperature=0.1,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=max_length,
    )
    return sequences[0]["generated_text"]

# -----------------------------
# 4) Run each prompt and print responses
# -----------------------------
print("**Combined (Zero-shot)**")
print(generate_response(prompt3_zero_shot, text_gen_pipeline))

print("\n**Combined (Chain-of-Thought)**")
print(generate_response(prompt3_cot, text_gen_pipeline))

# -----------------------------
# 5) Cleanup: Remove downloaded caches and free GPU memory
# -----------------------------
cleanup_all()

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**Combined (Zero-shot)**


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Analyze the following Java code for any security vulnerabilities and evaluate whether the function 'isValidRequest' correctly validates the request:
```java
import java.io.*;
import java.net.*;

public class ServerApp {
    public static void main(String[] args) {
        try {
            ServerSocket serverSocket = new ServerSocket(8080);
            while (true) {
                Socket socket = serverSocket.accept();
                Thread thread = new Thread(new RequestHandler(socket));
                thread.start();
            }
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}

class RequestHandler implements Runnable {
    private final Socket socket;

    public RequestHandler(Socket socket) {
        this.socket = socket;
    }

    // This method is intended to validate the incoming request.
    // It simply checks if the request starts with "GET".
    public static boolean isValidRequest(String request) {
        return request != null &

##Model 2 - Llama 2 7B Chat

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil
# -------------------------------------------------------------------
# 1) Model and Tokenizer Initialization for Llama2 7B Chat
# -------------------------------------------------------------------
# Define the model name (Hugging Face format for Llama2 7B Chat)
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model with 4-bit quantization enabled and automatic device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True
)

# -------------------------------------------------------------------
# 2) Helper Function for Generating Responses using Llama2 Chat
# -------------------------------------------------------------------
def generate_response_llama(prompt, max_new_tokens=1024):
    # Llama-2-Chat expects a conversation format. Simulate a single-turn conversation:
    # Format: <s>[INST] <<SYS>> system_message <</SYS>> user_message [/INST]
    system_msg = "You are a helpful security code analysis assistant."
    user_msg = prompt
    formatted_input = f"<s>[INST] <<SYS>>\n{system_msg}\n<</SYS>>\n\n{user_msg} [/INST]"

    # Tokenize the formatted input and move it to the same device as the model
    inputs = tokenizer(formatted_input, return_tensors='pt').to(model.device)

    # Generate the response with adjusted generation parameters:
    # - do_sample=True: enables sampling (controls randomness)
    # - temperature=0.2, top_p=0.9: parameters to control output diversity
    # - max_new_tokens=1024: sets the maximum length of the generated response
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.2,
        top_p=0.9
    )

    # Decode the generated tokens into a human-readable string, skipping special tokens
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# -------------------------------------------------------------------
# 3) Run Each Prompt and Print the Corresponding Responses
# -------------------------------------------------------------------
print("**Combined (Zero-shot)**")
print(generate_response_llama(prompt3_zero_shot))

print("\n**Combined (Chain-of-Thought)**")
print(generate_response_llama(prompt3_cot))


# -------------------------------------------------------------------
# 4) Cleanup: Freeing Up Memory Resources
# -------------------------------------------------------------------
cleanup_all()

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

**Combined (Zero-shot)**
[INST] <<SYS>>
You are a helpful security code analysis assistant.
<</SYS>>

Analyze the following Java code for any security vulnerabilities and evaluate whether the function 'isValidRequest' correctly validates the request:
```java
import java.io.*;
import java.net.*;

public class ServerApp {
    public static void main(String[] args) {
        try {
            ServerSocket serverSocket = new ServerSocket(8080);
            while (true) {
                Socket socket = serverSocket.accept();
                Thread thread = new Thread(new RequestHandler(socket));
                thread.start();
            }
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}

class RequestHandler implements Runnable {
    private final Socket socket;

    public RequestHandler(Socket socket) {
        this.socket = socket;
    }

    // This method is intended to validate the incoming request.
    // It simply checks if the request starts w

## Model 3 - StarCoder2 7B


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil
# -------------------------------------------------------------------
# 1) Model and Tokenizer Initialization for StarCoder2 7B
# -------------------------------------------------------------------
# Define the model name (Hugging Face format for StarCoder2 7B)
model_name = "bigcode/starcoder2-7b"

# Load the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model with 4-bit quantization enabled and automatic device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True
)

# -------------------------------------------------------------------
# 2) Helper Function for Generating Responses using StarCoder2
# -------------------------------------------------------------------
def generate_response_starcoder(prompt, max_new_tokens=1024):
    # Include the system message only once to set the context
    system_msg = "'''[System: You are an AI assistant skilled in code analysis.]'''"
    # Concatenate the system message with the user prompt
    formatted_input = f"{system_msg}\n{prompt}"

    # Tokenize the formatted input and move it to the model's device
    inputs = tokenizer(formatted_input, return_tensors='pt').to(model.device)

    # Generate the output using the model with specified generation parameters:
    # - do_sample=True: enables sampling for varied responses
    # - temperature=0.2: low temperature for controlled output
    # - top_p=0.9: nucleus sampling to limit the candidate pool
    # - pad_token_id and eos_token_id: ensure proper handling of tokenization for open-ended generation
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Decode the generated tokens to text, skipping special tokens
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------------------------------------------------------
# 3) Run Each Prompt and Print the Corresponding Responses
# -------------------------------------------------------------------
print("**Combined (Zero-shot)**")
print(generate_response_starcoder(prompt3_zero_shot))

print("\n**Combined (Chain-of-Thought)**")
print(generate_response_starcoder(prompt3_cot))

# -------------------------------------------------------------------
# 4) Cleanup: Freeing Up Memory Resources
# -------------------------------------------------------------------
cleanup_all()

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

**Combined (Zero-shot)**


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


'''[System: You are an AI assistant skilled in code analysis.]'''
Analyze the following Java code for any security vulnerabilities and evaluate whether the function 'isValidRequest' correctly validates the request:
```java
import java.io.*;
import java.net.*;

public class ServerApp {
    public static void main(String[] args) {
        try {
            ServerSocket serverSocket = new ServerSocket(8080);
            while (true) {
                Socket socket = serverSocket.accept();
                Thread thread = new Thread(new RequestHandler(socket));
                thread.start();
            }
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}

class RequestHandler implements Runnable {
    private final Socket socket;

    public RequestHandler(Socket socket) {
        this.socket = socket;
    }

    // This method is intended to validate the incoming request.
    // It simply checks if the request starts with "GET".
    public static boolean

## Model 4 - WizardCoder 13B
### Sometimes this causes issues where the GPU's RAM fills up. In this case, please restart the runtime and reload all the required Packages



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil


# -------------------------------------------------------------------
# 1) Model and Tokenizer Initialization for WizardCoder-Python 13B
# -------------------------------------------------------------------
# Define the model name (Hugging Face format for WizardCoder-Python 13B)
model_name = "WizardLMTeam/WizardCoder-Python-13B-V1.0"

# Load the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a BitsAndBytesConfig object to replace the deprecated load_in_4bit argument.
# This configuration enables 4-bit quantization and sets additional parameters:
# - llm_int8_enable_fp32_cpu_offload: Offload some modules to the CPU if needed.
# - bnb_4bit_use_double_quant: Uses double quantization for improved accuracy.
# - bnb_4bit_quant_type: Specifies the quantization type ("nf4" or "fp4").
# - bnb_4bit_compute_dtype: Sets the computation data type to float16 for performance.
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_enable_fp32_cpu_offload=True,  # Offload some modules to the CPU if needed.
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",             # You can also try "fp4" if needed.
    bnb_4bit_compute_dtype=torch.float16  # Using float16 for computation.
)

# Load the model with 4-bit quantization enabled and automatic device mapping.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True
)


# -------------------------------------------------------------------
# 2) Helper Function for Generating Responses using WizardCoder-Python
# -------------------------------------------------------------------
def generate_response_wizard(prompt, max_new_tokens=1024):
    # Tokenize the input prompt and move the resulting tensors to the model's device.
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    # Ensure input_ids are of type long
    inputs['input_ids'] = inputs['input_ids'].type(torch.long)

    # Generate the output with the following generation parameters:
    # - do_sample=False: disables sampling to produce deterministic output.
    # - temperature=0.2: controls the randomness of the generated output.
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        #do_sample=False,    # Change to True if you want sampling-based generation.
        temperature=0.2
    )

    # Decode the generated tokens into a human-readable string, skipping special tokens.
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------------------------------------------------------
# 3) Run Each Prompt and Print the Corresponding Responses
# -------------------------------------------------------------------
print("**Combined (Zero-shot)**")
print(generate_response_wizard(prompt3_zero_shot))

print("\n**Combined (Chain-of-Thought)**")
print(generate_response_wizard(prompt3_cot))

# -------------------------------------------------------------------
# 4) Cleanup: Remove Model and Tokenizer from Memory
# -------------------------------------------------------------------
cleanup_all()

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**Combined (Zero-shot)**


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Analyze the following Java code for any security vulnerabilities and evaluate whether the function 'isValidRequest' correctly validates the request:
```java
import java.io.*;
import java.net.*;

public class ServerApp {
    public static void main(String[] args) {
        try {
            ServerSocket serverSocket = new ServerSocket(8080);
            while (true) {
                Socket socket = serverSocket.accept();
                Thread thread = new Thread(new RequestHandler(socket));
                thread.start();
            }
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}

class RequestHandler implements Runnable {
    private final Socket socket;

    public RequestHandler(Socket socket) {
        this.socket = socket;
    }

    // This method is intended to validate the incoming request.
    // It simply checks if the request starts with "GET".
    public static boolean isValidRequest(String request) {
        return request != null &

## Model 5 - Mistral 7B Instruct


In [ ]:
# -------------------------------------------------------------------
# 1) Model and Tokenizer Initialization for Mistral-7B-Instruct
# -------------------------------------------------------------------
# Define the model name (Hugging Face format for Mistral-7B-Instruct v0.1)
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the tokenizer for the specified model.
# Note: use_fast is set to False to use the standard (non-fast) tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Load the model with 4-bit quantization enabled, automatic device mapping,
# and trusting remote code (if necessary for custom model implementations).
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

# -------------------------------------------------------------------
# 2) Helper Functions for Formatting the Prompt and Generating Responses
# -------------------------------------------------------------------
def format_mistral_prompt(user_prompt):
    return f"<s>[INST] {user_prompt} [/INST]"

def generate_response_mistral(user_prompt, max_new_tokens=1024):
    # Format the prompt as required by Mistral Instruct.
    prompt = format_mistral_prompt(user_prompt)

    # Tokenize the formatted prompt and move tensors to the model's device.
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    # Generate the output with specified parameters:
    # - do_sample=False: deterministic output.
    # - temperature=0.2: controls the randomness of generation.
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.2
    )

    # Decode the generated tokens into a human-readable string, skipping special tokens.
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------------------------------------------------------
# 3) Run Each Prompt and Print the Corresponding Responses
# -------------------------------------------------------------------
print("**Combined (Zero-shot)**")
print(generate_response_mistral(prompt3_zero_shot))

print("\n**Combined (Chain-of-Thought)**")
print(generate_response_mistral(prompt3_cot))

# -------------------------------------------------------------------
# 4) Cleanup: Remove Model and Tokenizer from Memory
# -------------------------------------------------------------------
cleanup_all()

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**Combined (Zero-shot)**


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Analyze the following Java code for any security vulnerabilities and evaluate whether the function 'isValidRequest' correctly validates the request:
```java
import java.io.*;
import java.net.*;

public class ServerApp {
    public static void main(String[] args) {
        try {
            ServerSocket serverSocket = new ServerSocket(8080);
            while (true) {
                Socket socket = serverSocket.accept();
                Thread thread = new Thread(new RequestHandler(socket));
                thread.start();
            }
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}

class RequestHandler implements Runnable {
    private final Socket socket;

    public RequestHandler(Socket socket) {
        this.socket = socket;
    }

    // This method is intended to validate the incoming request.
    // It simply checks if the request starts with "GET".
    public static boolean isValidRequest(String request) {
        return request !=